In [321]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/subodhjain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [322]:
dataset_train = pd.read_csv("train.csv")
dataset_test = pd.read_csv("test.csv")

In [323]:
dataset_train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [324]:
dataset_test.sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [325]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [326]:
dataset_train.dropna(inplace=True)
dataset_train.reset_index(drop=True)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [327]:
dataset_test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [328]:
X = dataset_train.selected_text.values
y = dataset_train.sentiment

In [329]:
X.shape

(27480,)

In [330]:
sentiment_mapper ={"negative":-1,"neutral":0,"positive":1}

y = y.map(sentiment_mapper)

In [331]:
# X = X[:200]
# y = y[:200]

In [332]:
y

0        0
1       -1
2       -1
3       -1
4       -1
        ..
27476   -1
27477   -1
27478    1
27479    1
27480    0
Name: sentiment, Length: 27480, dtype: int64

In [333]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

corpus=[]
for i in range(len(X)):
    tweet = re.sub("[^a-zA-Z]", " ", X[i])
    tweet = tweet.lower()
    tweet = tweet.split()
    t = [lemm.lemmatize(word) for word in tweet if not word in stopwords.words("english")]
    t = " ".join(t)
    corpus.append(t)
corpus[:20]

['responded going',
 'sooo sad',
 'bullying',
 'leave alone',
 'son',
 'http www dothebouncy com smf shameless plugging best ranger forum earth',
 'fun',
 'soooo high',
 '',
 'wow u became cooler',
 'much love hopeful reckon chance minimal p never gonna get cake stuff',
 'like',
 'dangerously',
 'lost',
 'test test lg env',
 'uh oh sunburned',
 'sigh',
 'sick',
 'onna',
 'he']

In [334]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features=10000, ngram_range=(1,2))
# X = cv.fit_transform(corpus).toarray()

In [335]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X = tfidf.fit_transform(corpus).toarray()

In [336]:
type(X)

numpy.ndarray

# Model building

In [337]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=20)

In [338]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [339]:
nb.score(X_train,y_train)

0.8393579168688339

In [341]:
nb.score(X_valid,y_valid)

0.8056768558951966